## CLC - Lab 4

### Task 4.1

In [1]:
import tweepy
from tweepy import OAuthHandler

consumer_key = '___________________'
consumer_secret = '______________________________________'
access_token = '______________________________________'
access_secret = '___________________'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)
user = api.me()

print('Name: ' + user.name)
# print 'Name: {}'.format(user.name)
# print('Location: ' + user.location)
# print('Friends: ' + str(user.followers_count))
# print('Created: ' + str(user.created_at))
# print('Description: ' + str(user.description))
print "\nTweets received:"

from tweepy import Stream
from tweepy.streaming import StreamListener
 
class MyListener(StreamListener):
    
    def on_data(self, data):
        try:
            with open('ArtificialIntelligenceTweets.json', 'a') as f:
                f.write(data)
                global tweetcount
                tweetcount += 1
                if not(tweetcount%10):
                    print " +"
                else:
                    print " +",
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True
 
tweetcount = 0
twitter_stream = Stream(auth, MyListener())
twitter_stream.filter(track=['ArtificialIntelligence','Artificial Intelligence'])


Name: Cedric Bhihe

Tweets received:
 +  +  +

KeyboardInterrupt: 

In [13]:
# -*- coding: utf-8 -*-
import nltk
import string
import operator 
import json
from nltk.corpus import stopwords
nltk.download("stopwords") # download the stopword corpus on our computer
from collections import Counter
import re
 
emoticons_str = r"""
    (?:
        [<>]?
        [:;=8]                          # eyes
        [\-o\*\'-]?                     # optional nose
        [\)\]\(\[dDpP/\:\>\<\}\{@\|\\]  # mouth
        |
        [\)\]\(\[dDpP/\:\>\<\}\{@\|\\]  # mouth
        [\-o\*\'-]?                     # optional nose
        [:;=8]                          # eyes
        [<>]?
        |
        <3                              # heart
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    # r'([a-zA-Z0-9_%\.-]+@[a-zA-Z0-9\.-]+\.[a-zA-Z0-9\.-]+)',   # email addresses
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.UNICODE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=True):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

# import io
# f=io.open('ArtificialIntelligenceTweets.json', 'r', encoding='utf8' )
with open('ArtificialIntelligenceTweets.json', 'r') as f:
    # "with" above entails that subsequent lines are indented.
    for line in f:
        tweet = json.loads(line)
        tokens = preprocess(tweet['text'])
        # print(tokens)
        

[nltk_data] Downloading package stopwords to /home/ckb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
language = "english"
punctuation = list(string.punctuation)
stop = stopwords.words(language) + punctuation + ['rt', 'via', 'RT', u'…']  # added ellipsis in utf-8 format.
# añadir '…' above in stop list, does not suppress detection of that token. 
 
fname = 'ArtificialIntelligenceTweets.json'
with open(fname, 'r') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
        # Create a list with all the terms
        terms_stop =   terms_only = [term for term in preprocess(tweet['text']) \
                                     if term not in stop and not term.startswith(('#', '@'))]
        # Mind the double brackets (( )) startswith() takes a tuple (not a list) if we pass a list of inputs.
        count_all.update(terms_stop)
    for word, index in count_all.most_common(5):
        print '%s : %s' % (word, index)

intelligence : 56
artificial : 54
apple : 12
acquired : 11
startup : 11


#### *Prepares (token, token usage frequency) value pairs to be plotted via Python simple HTTP server*

In [16]:
import vincent
 
word_freq = count_all.most_common(16)
labels, freq = zip(*word_freq)     # unzip list of pair values
data = {'data': freq, 'x': labels}
bar = vincent.Bar(data, iter_idx='x')
bar.to_json('term_freq.json')   # write the Vega spec to file 'term_freq.json'. If path not provided as 
                                # in 'bar.to_json()', write JSON as large single string to REPL.
                                # A simple HTML template to read and display the chart is built into Vincent. 
                                # It can be output along with the JSON:
                                # bar.to_json('term_freq.json', html_out=True, html_path='term_freq_template.html')
                                # HTML needs to be served. Start Python's simple HTTP Server, then point browser
                                # to localhost:8000: with '$ python -m SimpleHTTPServer 8000'

### Task 4.2: Create GeoJSON data structure for tweets

In [21]:
import json

fname = 'CaseStudy.json'
with open(fname, 'r') as f1:
    geo_data = {
        "type": "FeatureCollection",
        "features": []
    }
    for line in f1:
        tweet = json.loads(line)
        if tweet['coordinates']:
            geo_json_feature = {
                "type": "Feature",
                "geometry": tweet['coordinates'],
                "properties": {
                    "text": tweet['text'],
                    "created_at": tweet['created_at']
                }
            }
            geo_data['features'].append(geo_json_feature)
 
with open('geo_data.json', 'w') as fout:
    fout.write(json.dumps(geo_data, indent=4))

### Task 4.3: Apply to barcelona.json *(lab4.StudentProposal_bhihe)*

In [1]:
import tweepy
from tweepy import OAuthHandler

# consumer_key = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
# consumer_secret = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
# access_token = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
# access_secret = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)
user = api.me()

print('Tweeter Account Owner: ' + user.name)


Tweeter Account Owner: Cedric Bhihe


In [4]:
import json
from tweepy import Stream
from tweepy.streaming import StreamListener
 
class BarcelonaListener(StreamListener):
 
    def on_data(self, data):
        try:
            with open('barcelona.json', 'a') as f:
                f.write(data)
                global tot_tweetcount
                tot_tweetcount += 1
                if not(tot_tweetcount%40):
                    print " +"
                else:
                    print " +",
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True
 
tot_tweetcount = 0
twitter_stream = Stream(auth, BarcelonaListener())
twitter_stream.filter(track=['Barcelona', 'BCN'])

  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +
 +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +
 +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +  +
 +  +  +

KeyboardInterrupt: 

In [5]:
fname = 'barcelona.json'
global tweet_count
tweet_count = 0

with open(fname, 'r') as f2:
    geo_data = {
        "type": "FeatureCollection",
        "features": []
    }
    for line in f2:
        tweet = json.loads(line)
        if tweet['coordinates']:
            tweet_count +=1
            geo_json_feature = {
                "type": "Feature",
                "geometry": tweet['coordinates'],
                "properties": {
                    "text": tweet['text'],
                    "created_at": tweet['created_at']
                }
            }
            geo_data['features'].append(geo_json_feature)
 
with open('geo_data.json', 'w') as fout:
    fout.write(json.dumps(geo_data, indent=4))
    
print "Conclusion:"
print "  - {} out of {} tweets could be geolocalized.".format(tweet_count,tot_tweet_count)

 Conclusion:
  - 5 out of 123 tweets could be geolocalized.


### **Conclusion:** 
- _Out of 123 tweets captured from a stream tracking two keywords ("Barcelona" and "BCN"), only 5 tweets were geolocalized._
- _Results are mapped in the file barcelona.html._